In [7]:
import os
import glob
import pandas as pd
import textgrids
import tqdm
from utils import flac_to_wav

# Create Subset json

## Keep files in subset

In [53]:
import ast
import pandas as pd

def genorate_pps_df(json_dir:str):
    with open(json_dir, 'r') as json_file:
        json_list = list(json_file)

    df = pd.DataFrame.from_dict(json_list)
    df[0] = df[0].apply(lambda x: ast.literal_eval(x))

    for keys in df[0][0].keys():
        df[keys] = [path[0][keys] for path in df.iloc()]
    df = df.drop(0, axis=1)

    df = df[['training_data']] # Keep only columbs that are needed to split
    for keys in df['training_data'][0].keys():
        df[keys] = [path[0][keys] for path in df.iloc()]
    df = df[['label', 'name']] # Keep only columbs that are needed to split
    #audio_path, duration, shard_id, text

    subset_df = pd.DataFrame()

    labels = []
    names = []

    for row in df.iloc():
        label, name = row
        for l, n in zip(label, name):
            labels.append(l)
            names.append(n)
    
    subset_df['audio_path'] = names
    subset_df['text'] = labels

    return subset_df
names=['audio_path', 'text']
names=None

pps_df = pd.read_csv('/home/knoriy/split_peoples_speech/pps_train.tsv', sep='\t', header=None, names=names)

In [54]:
pps_df

,0,1
0,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,is there anything to wear it on this one or ha...
1,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,i thought the law was that we don't have juris...
2,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,review it yes you do yes you do
3,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,this you know what happened there was a single...
4,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,motion to abandon the quantity because nothing...
...,...,...
4320997,The_Capture_movie/Capture_DOT_mp3_00011.flac,reward of all our marriage
4320998,The_Capture_movie/Capture_DOT_mp3_00012.flac,so i traveled back the way i'd come on it this...
4320999,The_Capture_movie/Capture_DOT_mp3_00013.flac,you state your business mister
4321000,The_Capture_movie/Capture_DOT_mp3_00014.flac,come out yet i don't want to be just that


In [55]:
def get_subset_df(dataset_root_path:str, df:pd.DataFrame):
    subset = glob.glob(dataset_root_path, recursive=True)
    subset = [os.path.join(*(dir.split(os.path.sep)[5:])) for dir in subset]

    subset_df = df[df['audio_filepath'].isin(subset)].reset_index(drop=True)
    subset_df = subset_df.drop(0, axis=1)

    return subset_df

root_path = '/home/knoriy/split_peoples_speech/'
dataset_name = 'pps_train'
dataset_root_path = os.path.join(root_path, f'{dataset_name}')

import glob

df = get_subset_df(f'{dataset_root_path}/**/*.flac', pps_df)
df


,1
0,members let us return to order that the it is ...
1,would ask mr buzz if he has any comments to sh...
2,its community development block grant program ...
3,patient our early on the projects that we've s...
4,thank you very much that's all chair thank yo...
...,...
95,cameras resist this set aside for future purch...
96,alarm system type components that are necessar...
97,that needed in your department chairman period...
98,don't have their own internal staff to be able...


# prepare MFA

In [32]:
# Clean old wavs ONLY FOR TESTING
import glob

old_wav_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/mini_subset/**/*.wav', recursive=True)
for wav_path in old_wav_path:
    os.remove(wav_path)

# old_txt_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/subset/**/*.txt', recursive=True)
# for txt_path in old_txt_path:
#     os.remove(txt_path)

# old_txt_path = glob.glob(f'/home/knoriy/Documents/laion/split_peoples_speech/subset_split/**/*.txt', recursive=True)
# for txt_path in old_txt_path:
#     os.remove(txt_path)

# Tar loader

# ToDo
- [ ] Download and extract single or batch of files from aws
- [ ] Process for MFA
- [ ] Generate textgrid
- [ ] Split audio and upload to aws s3

# AWS S3

In [ ]:
%%bash

nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-07a8f0d3-6d27-4299-887a-dc12a6d72f8d-c000.tar?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A30%3A22Z/-1/host/444b9c082ceffd10f38bb965679ed9ec12202836831e111dd193fde281062d26 -O - | aws s3 cp - s3://s-laion/peoples_speech/train_clean_pps.tar" &;
nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-4e132642-c01c-4db6-9db0-a1e19193f6f8-c000.json?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A31%3A22Z/-1/host/d7dacf3c31d2e3670d82727636ce234be27a9128df7a80883b84b4a3d8c7f6c0 -O - | aws s3 cp - s3://s-laion/peoples_speech/Manifest.json" &;


In [ ]:
%%bash

nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/mswc.tar.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/full_mswc.tar" > full_mswc.out &
nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/metadata.json.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/metadata.tar" > mswc_meta.out &



In [ ]:
%%bash

# nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/mswc.tar.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/full_mswc.tar" > full_mswc.out &
# nohup sh -c "wget https://storage.googleapis.com/public-datasets-mswc/metadata.json.gz -O - | aws s3 cp - s3://s-laion/multilingual_spoken_words/metadata.tar" > mswc_meta.out &



In [30]:
import tarfile

metadata_dir = "/mnt/knoriy/metadata.json"
tar_dir = "/mnt/knoriy/pps_train.tar"

with tarfile.open(tar_dir, mode='r') as src_file_obj:
    print('opening file: This may take some time\n')

    file_names_full_list = src_file_obj.getnames()
    file_names_full_list = [i for i in file_names_full_list if '.flac' in i]

opening file: This may take some time



In [31]:
file_names_full_list[0]

'CC20160404BudgetFinance2of2/CC-2016-0404-BudgetFinance_2of2_DOT_mp3_00000.flac'